In [1]:
import pandas as pd
import numpy as np

In [2]:
#df_abt = pd.read_csv("output/allcruises_df_validated_5with_zoomie.csv")
df_abt = pd.read_csv('output/update_allcruises_df_validated_5with_zoomie_20230727.csv',sep=";")
df_abt =df_abt[df_abt["label"] != "Artefact"]
#test = pd.read_csv("output/update_wo_artefacts_test_dataset_PS992_03032023.csv")
test = pd.read_csv("output/update_wo_artefacts_test_dataset_PS992_20230727_nicole.csv",sep=";")


In [14]:
list(df_abt.columns)

['new_index',
 'img_file_name',
 'img_rank',
 'object_id',
 'object_lat',
 'object_lon',
 'object_date',
 'object_time',
 'object_link',
 'object_depth_min',
 'object_depth_max',
 'object_annotation_status',
 'object_annotation_person_name',
 'object_annotation_person_email',
 'object_annotation_date',
 'object_annotation_time',
 'object_annotation_category',
 'object_annotation_category_id',
 'object_index',
 'object_haul',
 'object_bottom_depth',
 'object_pressure',
 'object_temperature',
 'object_salinity',
 'object_conductivity',
 'object_oxygen_concentration',
 'object_temperature_oxsens',
 'object_oxygen_saturation',
 'object_dr._haardt_fluorescence_channel_a',
 'object_area_px',
 'object_form',
 'object_area',
 'object_length',
 'object_width',
 'object_convexity',
 'object_structure',
 'object_graymean',
 'object_kurtosis',
 'object_skewness',
 'object_hu_moment_1',
 'object_hu_moment_2',
 'object_hu_moment_3',
 'object_hu_moment_4',
 'object_hu_moment_5',
 'object_hu_moment_6'

0         2.0
1         0.0
2         1.0
3         0.0
4         0.0
         ... 
224721    5.0
224722    5.0
224723    0.0
224724    2.0
224725    4.0
Name: object_area, Length: 217378, dtype: float64

In [21]:
df_count_deploy = pd.DataFrame(df_abt["object_station"].value_counts())

In [25]:
df_count_deploy["Cruise_Number_"] =df_count_deploy.index

In [27]:
df_count_deploy["Cruise_Number"] =df_count_deploy["Cruise_Number_"].apply(lambda x: x.split("-")[0])

In [35]:
df_count_deploy["Cruise_Number_clean"] = df_count_deploy["Cruise_Number"].apply(lambda x: x.split("_")[0])

In [38]:
df_count_deploy["Cruise_Number_clean_2"]= df_count_deploy["Cruise_Number_clean"].apply(lambda x: x.split(".")[0])

In [39]:
df_count_deploy["Cruise_Number_clean_2"].value_counts()

PS106    20
PS122    19
PS107    17
PS99      4
PS114     1
Name: Cruise_Number_clean_2, dtype: int64

PS107      121628
PS106.2     42462
PS99.2      22899
PS122        8194
PS114        7199
PS122.1      7075
PS122-2      6495
PS122.2      1426
Name: object_cruise, dtype: int64

In [3]:
df_abt_tv =df_abt[df_abt["object_cruise"] != "PS99.2"]
df_abt_tv_S = df_abt_tv[["label","count"]].groupby("label").sum()
df_abt_t_S = test[["label","count"]].groupby("label").sum()
df_abt_tv_S = df_abt_tv_S.rename(columns={"count": "train_val"})
df_abt_t_S =df_abt_t_S.rename(columns={"count": "test"})
df_plot = df_abt_tv_S.join(df_abt_t_S, on="label")
df_plot.to_csv("paper/tables/20230727_train_val_test.csv", sep=";")

In [4]:
df_abt_99 =test[test["object_cruise"] == "PS99.2"]
df_abt_106 =df_abt[df_abt["object_cruise"] == "PS106.2"]
df_abt_107 =df_abt[df_abt["object_cruise"] == "PS107"]
df_abt_114 =df_abt[df_abt["object_cruise"] == "PS114"]
df_abt_122 =df_abt[df_abt["object_cruise"].isin(['PS122', 'PS122-2', 'PS122.1', 'PS122.2'])]

In [5]:
all_cls = list(np.unique(df_abt["label"]))

In [6]:
dl_99 = df_abt_99[["label","count"]].groupby("label").sum()
dl_106 = df_abt_106[["label","count"]].groupby("label").sum()
dl_107 = df_abt_107[["label","count"]].groupby("label").sum()
dl_114 = df_abt_114[["label","count"]].groupby("label").sum()
dl_122 = df_abt_122[["label","count"]].groupby("label").sum()

In [7]:
for i in [dl_99, dl_106, dl_107, dl_114, dl_122]:
    print(len(i))
temp = dl_106.join(dl_99, on="label", rsuffix='_99')
temp = temp.join(dl_107, on="label", rsuffix='_107')
temp = temp.join(dl_114, on="label", rsuffix='_114')
temp = temp.join(dl_122, on="label",lsuffix='_106', rsuffix='_122')
temp['sum'] = temp[['count_106', 'count_99', 'count_107', 'count_114', 'count_122']].sum(axis=1)
temp.to_csv("paper/tables/20230727_statistics_over_cruises.csv", sep=";")

31
33
32
25
30


In [ ]:
from matplotlib import pyplot as plt
fig, axes = plt.subplots(nrows=5, ncols=1)
cruises = ["PS99.2", "PS106.2", "PS107", "PS114", "PS122"]
for index, df in enumerate([df_abt_99, df_abt_106, df_abt_107, df_abt_114, df_abt_122]):
    #print(df["object_cruise"].values[0])
    df = df[["label", "count"]]
    df=df.groupby("label").sum()
    sum1 = df.sum().item()
    df["count"] =df["count"]/sum1
    #print(df.sum().item())
    df.plot(ax =axes[index],  width=1.0, figsize=(25, 50), kind='bar')
    axes[index].set_ylabel('Number of Sample in % of total Samples')
    axes[index].set_title(f'{cruises[index]} #Samples={sum1}')
plt.tight_layout()
plt.savefig(f'paper/figures/20230727_CruiseStats.jpg')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
a = df_abt_99[['label','count']].groupby('label').agg(sum)/df_abt_99[['label','count']].groupby('label').agg(sum).sum()
b = df_abt_106[['label','count']].groupby('label').agg(sum)/df_abt_106[['label','count']].groupby('label').agg(sum).sum()
c = df_abt_107[['label','count']].groupby('label').agg(sum)/df_abt_107[['label','count']].groupby('label').agg(sum).sum()
d = df_abt_114[['label','count']].groupby('label').agg(sum)/df_abt_114[['label','count']].groupby('label').agg(sum).sum()
e = df_abt_122[['label','count']].groupby('label').agg(sum)/df_abt_122[['label','count']].groupby('label').agg(sum).sum()

#t = cosine_similarity(X=d.values, Y=c.values, dense_output=True)
a_new = b.join(a, rsuffix='_PS99').drop('count', axis=1).fillna(0)
c_new = b.join(c, rsuffix='_PS107').drop('count', axis=1).fillna(0)
d_new = b.join(d, rsuffix='_PS114').drop('count', axis=1).fillna(0)
e_new = b.join(e, rsuffix='_PS122').drop('count', axis=1).fillna(0)
#t = cosine_similarity(X=a_new, Y=c_new, dense_output=True)
b_new =b
b_new = b_new.rename(columns={'count':'count_PS106'})
b_new = b_new.join(a_new)
b_new = b_new.join(c_new)
b_new = b_new.join(d_new)
b_new = b_new.join(e_new)
b_new.corr()

b_new['PS_train_val'] =  b_new['count_PS122'] + b_new['count_PS106'] + b_new['count_PS107'] +b_new['count_PS114']
b_new['PS_all'] = b_new['count_PS122'] + b_new['count_PS106'] + b_new['count_PS107'] +b_new['count_PS114'] + b_new['count_PS99']
#plt.imshow(b_new.corr())
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

ax = sns.heatmap(b_new.corr(), linewidth=0.5, cmap='viridis')
#plt.show()
plt.tight_layout()
plt.savefig(f'paper/figures/20230727_HeatmapCorrelationLabelDistribution.jpg')
plt.show()

In [ ]:
b_new

In [ ]:
import matplotlib.pyplot as plt
# Pie chart
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
MEDIUM_SIZE = 18
BIGGER_SIZE = 18
SMALL_SIZE=18
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
#r_new =  b_new[['count_PS99']].sort_values("count_PS99")
#r_plot = r_new[r_new['count_PS99']>= 0.00]
#r_rest = r_new[r_new['count_PS99']<= 0.00]
#print(r_rest.T)
#print(r_rest.sum())
#r_plot = r_plot.T
#r_plot['other'] =r_rest.sum().values
#r_plot = r_plot.T
#r_plot = r_plot['count_PS99']
#r_new.loc[r_new[['count_PS99']] < 0.05, 'label'] = 'rest'
#r_new = r_new.groupby('label')['count_PS99'].sum().reset_index()
b_new = b_new.sort_values("PS_all").tail(10)
print(b_new)
labels = list(b_new.index)
sizes = b_new['count_PS99'].values
sizes0 = b_new['count_PS106'].values
sizes1 =b_new['count_PS107'].values
sizes2 =b_new['count_PS114'].values
sizes3 =b_new['count_PS122'].values

#colors
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']
color_new = cm.get_cmap('rainbow', len(labels))
newcolors = color_new(np.linspace(0, 1, len(labels)))

fig1, ax1 = plt.subplots(figsize=(16,9))
#ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=25)
#ax1 = fig.add_axes([0,0,1,1])
x = np.arange(len(labels))

ax1.bar(x +0.00, height=sizes, color = 'black', width = 0.2)
ax1.bar(x +0.20, height=sizes0, color = 'coral', width = 0.2)
ax1.bar(x +0.40, height=sizes1, color = 'lightgreen', width = 0.2)
ax1.bar(x +0.60, height=sizes2, color = 'steelblue', width = 0.2)
ax1.bar(x +0.80, height=sizes3, color = 'plum', width = 0.2)

plt.legend(["count_PS99", "count_PS106", "count_PS107", "count_PS114", "count_PS122"])
#draw circle
#centre_circle = plt.Circle((0,0),0.70,fc='white')
#fig = plt.gcf()
#fig.gca().add_artist(centre_circle)
#ax1.set_title('HI')
# Equal aspect ratio ensures that pie is drawn as a circle
#ax1.axis('equal')
#ax1.set_xticklabels(x, labels, rotation=90)
plt.xticks(x, labels,rotation=90)
plt.tight_layout(pad=3.95, w_pad=3.95, h_pad=3.0)
plt.grid(visible=False)
plt.savefig(f'paper/figures/20230727_Plot.jpg')
plt.show()